In [9]:
from config import settings
from src.partition import Partition
from src.generators import PartitionClass
import numpy as np
import os

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score

from keras.models import Sequential, Model, load_model
from keras.layers import (
    Dense, Conv2D, Flatten, 
    ConvLSTM2D, BatchNormalization,
    Conv3D, Input, MaxPooling2D, UpSampling2D,
)

from keras.callbacks.callbacks import EarlyStopping, ReduceLROnPlateau

In [ ]:
settings.model_folder_path

# Keras Stability Model

In [2]:
n = 50

In [3]:
training_partitions = [p for p in PartitionClass(n).grid_group(n)]
training_matrices = [p.fit_matrix(n) for p in training_partitions]
training_targets = [p.is_stable for p in training_partitions]

X = np.array(training_matrices)
y = np.array(training_targets)

label_binarizer = LabelBinarizer()
y = label_binarizer.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X, y)

X_train = X_train.reshape(len(X_train), n, n, 1)
X_test = X_test.reshape(len(X_test), n, n, 1)

In [4]:
#create model
stability_model = Sequential()

#add model layers
stability_model.add(Conv2D(64, kernel_size=3, activation="relu", input_shape=(n, n, 1)))
stability_model.add(MaxPooling2D((2, 2), padding='same'))
stability_model.add(Conv2D(32, kernel_size=2, activation="relu"))
stability_model.add(MaxPooling2D((2, 2), padding='same'))
stability_model.add(Conv2D(16, kernel_size=2, activation="relu"))
stability_model.add(MaxPooling2D((2, 2), padding='same'))
stability_model.add(Flatten())
stability_model.add(Dense(units=500, activation='relu'))
stability_model.add(Dense(units=100, activation='relu'))
stability_model.add(Dense(units=20, activation='relu'))
stability_model.add(Dense(units=len(y[0]), activation='sigmoid'))

stability_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

W1016 20:26:10.847763 4543968704 deprecation_wrapper.py:119] From /Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4070: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W1016 20:26:11.050448 4543968704 deprecation.py:323] From /Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/tensorflow/python/ops/nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [6]:
lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.001)
es = EarlyStopping(monitor='val_loss', min_delta=.001, patience=5, verbose=0, mode='auto', baseline=None, restore_best_weights=False)
cbs = [es, lr]

In [7]:
stability_model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=1, batch_size=1000, workers = 4)

Train on 971977 samples, validate on 323993 samples
Epoch 1/1
971977/971977 [==============================] - 1339s 1ms/step - loss: 4.7093e-07 - accuracy: 1.0000 - val_loss: 1.6107e-07 - val_accuracy: 1.0000


In [11]:
model_path = os.path.join(settings.model_folder_path, 'stability.h5')
# Serialize it
stability_model.save(model_path)
del stability_model
stability_model = load_model(model_path)

In [12]:
# 100% accuracy
print(f'Accuracy: {stability_model.evaluate(X_test, y_test, verbose = False)[1] * 100}%')
print(f'AUC Score: {roc_auc_score(y_test, stability_model.predict_proba(X_test)) * 100}%')

Accuracy: 100.0%
AUC Score: 100.0%
